In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
# load parquet
PARQUET_PATH = '../base/base_de_dados.parquet'

df = pd.read_parquet(PARQUET_PATH)
df = df[
     [
        'id',
        'approved',
        'created_at',
        'analysis_status',
        'kind',
        'finished'
     ]]
     
df = df[df['finished'] == True].drop(columns=['finished'], inplace=False)

#convert created_at to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
df['month'] = df['created_at'].dt.month
df = df.drop(columns=['created_at'], inplace=False)

In [68]:
# download all .jpeg files from base-teste-facematch-analise-dispositivo-dev S3
# and save to local folder
import boto3
import os
import tqdm

# Initialize a session using Amazon S3
s3 = boto3.client('s3')

bucket_name = 'base-teste-facematch-analise-dispositivo-dev'

# Create a local directory to store the files
local_folder = '/home/joao.cassiano/Documentos/will/projetos/facematch_interno/base/imagens'
os.makedirs(local_folder, exist_ok=True)

# List and download files from the bucket
def download_jpeg_files():
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name):
        for obj in tqdm.tqdm(page.get('Contents', [])):
            key = obj['Key']
            if key.endswith('.jpeg'):
                local_file_path = os.path.join(local_folder, key)
                s3.download_file(bucket_name, key, local_file_path)
                #print(f'Downloaded {key} to {local_file_path}')

download_jpeg_files()


 23%|██▎       | 227/1000 [01:41<05:45,  2.24it/s]


KeyboardInterrupt: 

In [47]:
#get number of occurences of each status
df.analysis_status.value_counts()

analysis_status
BIOMETRIC_AUTHENTICATED              19447
SUSPECTED_FRAUD                       2995
BIOMETRIC_TRANSACTION_BAD_QUALITY      433
BIOMETRIC_NEEDS_REVALIDATION           372
UNREADABLE_PHOTO                       228
UNREADABLE_BIOMETRIC_FACE                2
Name: count, dtype: int64

In [48]:
df.head(10)

,id,user_id,approved,created_at,selfie_file_path,analysis_status,kind
0,c1f483cd-0ead-4c83-bd7f-2434e89cd7ba,630122aa-a176-4ae8-96ed-3fa7ac85a07c,True,2023-10-22 18:38:54.629499,private/630122aa-a176-4ae8-96ed-3fa7ac85a07c/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
1,81e61bb8-dcde-4334-a43c-a9d4d7b03a4b,2977d685-5e69-455f-97ab-9483496723ac,True,2023-10-22 18:43:57.934048,private/2977d685-5e69-455f-97ab-9483496723ac/u...,BIOMETRIC_TRANSACTION_BAD_QUALITY,MANUAL_BY_BIOMETRIC_FACE
2,9be3c172-d260-4def-a941-eb0e2761becd,5a2b7e70-5230-46f1-b2fc-057baacb3f7c,True,2023-09-16 07:01:41.390316,private/5a2b7e70-5230-46f1-b2fc-057baacb3f7c/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
3,83e57219-6fae-4bb7-8e67-2b0271fc1adb,370b4c9b-afd0-480a-91ba-30bc9fac32bf,True,2023-10-22 18:36:48.334290,private/370b4c9b-afd0-480a-91ba-30bc9fac32bf/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
4,e23886d9-755f-4b1c-a523-50561f6493a7,5eeaca8d-09cf-4dfd-9cd7-9740de4e3043,True,2023-10-22 18:39:54.001609,private/5eeaca8d-09cf-4dfd-9cd7-9740de4e3043/u...,BIOMETRIC_NEEDS_REVALIDATION,MANUAL_BY_BIOMETRIC_FACE
5,f6a47b32-c401-4fca-b945-7a70f63292dd,f78ddbfe-dfd5-4354-9c77-b7e749b366a0,True,2023-10-22 18:45:25.591490,private/f78ddbfe-dfd5-4354-9c77-b7e749b366a0/u...,BIOMETRIC_NEEDS_REVALIDATION,MANUAL_BY_BIOMETRIC_FACE
6,4f736972-1899-4d71-8d7f-c2a71ade52d7,1a96e865-77e3-4dc8-a6a1-9bd2e1cec052,True,2023-10-22 18:50:35.600521,private/1a96e865-77e3-4dc8-a6a1-9bd2e1cec052/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
7,a96fcd10-2e5c-45e4-b912-bba073e377b4,53aaa9eb-7e65-4882-a563-674d55f427d0,False,2023-09-16 09:07:43.088118,private/53aaa9eb-7e65-4882-a563-674d55f427d0/u...,SUSPECTED_FRAUD,MANUAL_BY_BIOMETRIC_FACE
8,4a52962e-e90d-4579-80c7-7ed8f392f5eb,c53f37df-70cd-4cdf-adc6-d29fdd1a6851,True,2023-09-17 14:16:24.602076,private/c53f37df-70cd-4cdf-adc6-d29fdd1a6851/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
9,fc1cd595-e85e-4a0a-9b58-f96c2dd95949,55aa2ce7-0337-437a-98b8-bc1b3b62b700,True,2023-10-22 18:39:20.744156,private/55aa2ce7-0337-437a-98b8-bc1b3b62b700/u...,BIOMETRIC_AUTHENTICATED,MANUAL_BY_BIOMETRIC_FACE
